# State Farm Distracted Driver Detection

In [19]:
# %pip install ultralytics
# %pip install split-folders
# %pip install -U ipywidgets

In [20]:
import csv
import os
from glob import glob
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from ultralytics import YOLO
import splitfolders
import pandas as pd
import numpy as np
from IPython.display import display, Image
from sklearn.metrics import accuracy_score

## Loading the data

In [21]:
data = {}

with open('../datasets/state-farm-distracted-driver-detection/driver_imgs_list.csv') as file:
    read_file = csv.reader(file)
    read_file = list(read_file)
    
    for row in read_file[1:]:
        key = row[1]
        if key in data:
            data[key].append(row[2])
        else:
            data[key] = [row[2]]

In [22]:
data['c0'][:5]

['img_44733.jpg',
 'img_72999.jpg',
 'img_25094.jpg',
 'img_69092.jpg',
 'img_92629.jpg']

In [23]:
classes_list = list(data.keys())
classes_list

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [24]:
dataset_folder = '../datasets/state-farm-distracted-driver-detection/imgs/'

train_dir = os.path.join(dataset_folder, 'train/')
test_dir = os.path.join(dataset_folder, 'test/')

In [25]:
print('Number of images in the training dataset : ', str(len(glob(train_dir+'*/*'))))
print('Number of images in the testing dataset : ', str(len(glob(test_dir+'*'))))

Number of images in the training dataset :  22424
Number of images in the testing dataset :  79726


In [ ]:
# Need to separate the training set and the validation set.

### Writing helper function for creating directories for training set, validation set and test set

In [26]:
def remove_directory(path):
    for root, dirs, files in os.walk(path, topdown = False):
        for name in files:
            file_path = os.path.join(root, name)
            os.remove(file_path)
        for name in dirs:
            dir_path = os.path.join(root, name)
            os.rmdir(dir_path)
    os.rmdir(path)

In [27]:
def create_directories(paths, subfolders):
    for path in paths:
        if os.path.exists(path):
            remove_directory(path)
        
        for folder in subfolders:
            subfolder_path = os.path.join(path, folder)
            os.makedirs(subfolder_path)

In [28]:
paths = ['../datasets/state-farm-distracted-driver-detection/cleaned_dataset/train',
         '../datasets/state-farm-distracted-driver-detection/cleaned_dataset/val',
        '../datasets/state-farm-distracted-driver-detection/cleaned_dataset/test']

In [29]:
subfolders = classes_list

### Creating Train, Val, Test folders along with sub-directories (all Classes)

In [31]:
create_directories(paths, subfolders)

### Creating the cleaned dataset using the above helper functions we have created

In [32]:
split_size = [0.6, 0.2]


for clas, images in data.items():
    # print(len(images))
    train_size = int(split_size[0]*len(images))
    # print("Train size: ", train_size)
    
    test_size = int(split_size[1]*len(images))
    #print("Test size: ", test_size)
    
    train_images = images[:train_size]
    # print("Train Images Length", len(train_images))
    
    val_images = images[train_size: train_size + test_size]
    # print("Val Images Length", len(val_images))
    
    test_images = images[train_size + test_size:]
    # print("Test Images Length", len(test_images))
    
    
    
    for image in train_images:
        source = os.path.join(train_dir, clas, image)
        # print(os.path.exists(source))
        dest = os.path.join(paths[0], clas, image)
        copyfile(source, dest)
    
    for image in val_images:
        source = os.path.join(train_dir, clas, image)
        dest = os.path.join(paths[1], clas, image)
        copyfile(source, dest)
    
    for image in test_images:
        source = os.path.join(train_dir, clas, image)
        dest = os.path.join(paths[2], clas, image)
        copyfile(source, dest)
    
    


### Using a better approach for creating the cleaned dataset using `splitfolders` module

### First deleting the cleaned dataset created usinfg the above method

In [33]:
remove_directory('../datasets/state-farm-distracted-driver-detection/cleaned_dataset')

### Creating the cleaned dataset now using splitfolder module

In [34]:
images_dir = '../datasets/state-farm-distracted-driver-detection/imgs/train'
output_folder = '../datasets/state-farm-distracted-driver-detection/cleaned_dataset' # Note: the function will create val, train, test sub directories by itself
split_ratio = (0.6, 0.2, 0.2)



splitfolders.ratio(images_dir, output= output_folder, seed = 10, ratio= split_ratio)

Copying files: 22424 files [00:03, 5949.75 files/s]


Done ! Just needed one line of code.

### From now, we will be using these Directory paths for our training, validation and testing purpose

In [35]:
parent_dir = '../datasets/state-farm-distracted-driver-detection/cleaned_dataset'
train_dir = '../datasets/state-farm-distracted-driver-detection/cleaned_dataset/train'
val_dir = '../datasets/state-farm-distracted-driver-detection/cleaned_dataset/val'
test_dir = '../datasets/state-farm-distracted-driver-detection/cleaned_dataset/test'

## Creating Image data generator Function with Data Augmentation

In [53]:
def imagedatageneration(train_dir, val_dir, test_dir, target_size = (256, 256), batch_size = 32):
    train_datagen = ImageDataGenerator(rescale = 1.0 / 255,
                                       rotation_range = 30,
                                       width_shift_range = 0.1,
                                       height_shift_range = 0.1,
                                       zoom_range = 0.1,
                                       shear_range = 0.1,
                                       fill_mode = "nearest"
                                      )
    train_generator = train_datagen.flow_from_directory(
                                                            train_dir,
                                                            target_size = target_size,
                                                            class_mode = 'categorical',
                                                            shuffle = True,
                                                            batch_size = batch_size
                                                        )
    
    
    val_datagen = ImageDataGenerator(rescale = 1.0 / 255,
                                     rotation_range = 30,
                                     width_shift_range = 0.1,
                                     height_shift_range = 0.1,
                                     zoom_range = 0.1,
                                     shear_range = 0.1,
                                     fill_mode = "nearest"
                                    )
    val_generator = val_datagen.flow_from_directory(
                                                        val_dir,
                                                        target_size = target_size,
                                                        class_mode = 'categorical',
                                                        shuffle = True,
                                                        batch_size = batch_size
                                                    )
    
    test_datagen = ImageDataGenerator(rescale = 1.0/255)
    
    test_generator = test_datagen.flow_from_directory(
                                                        test_dir,
                                                        target_size = target_size,
                                                        class_mode = 'categorical',
                                                        shuffle = False,
                                                        batch_size = 1
                                                      )
    
    return train_generator, val_generator, test_generator
    

In [54]:
es = EarlyStopping(monitor = 'val_acc', patience = 2, min_delta = 0.01)

## First Model -> Dense Model

In [55]:
train_generator, val_generator, test_generator = imagedatageneration(train_dir, val_dir, test_dir)

Found 13451 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.
Found 4492 images belonging to 10 classes.


In [56]:
model1 = tf.keras.models.Sequential([
    Flatten(input_shape = (256, 256, 3)),
    Dense(1024, activation = 'relu'),
    BatchNormalization(),
    Dense(512, activation = 'relu'),
    BatchNormalization(),
    Dense(256, activation = 'relu'),
    BatchNormalization(),
    Dense(10, activation = 'softmax')
])

In [58]:
model1.compile(optimizer = Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['acc'])

In [59]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 196608)            0         
                                                                 
 dense_4 (Dense)             (None, 1024)              201327616 
                                                                 
 batch_normalization_3 (Bat  (None, 1024)              4096      
 chNormalization)                                                
                                                                 
 dense_5 (Dense)             (None, 512)               524800    
                                                                 
 batch_normalization_4 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dense_6 (Dense)             (None, 256)              

In [60]:
model1.fit(train_generator,
                   epochs = 15,
                   verbose = 1,
                   validation_data = val_generator,
                   callbacks = [es])

Epoch 1/15
421/421 [==============================] - 130s 308ms/step - loss: 2.2450 - acc: 0.2062 - val_loss: 2.0980 - val_acc: 0.2486
Epoch 2/15
421/421 [==============================] - 129s 307ms/step - loss: 1.9323 - acc: 0.2980 - val_loss: 1.8307 - val_acc: 0.3330
Epoch 3/15
421/421 [==============================] - 130s 308ms/step - loss: 1.7795 - acc: 0.3521 - val_loss: 1.6930 - val_acc: 0.3879
Epoch 4/15
421/421 [==============================] - 129s 307ms/step - loss: 1.6590 - acc: 0.4014 - val_loss: 1.6717 - val_acc: 0.4008
Epoch 5/15
421/421 [==============================] - 130s 308ms/step - loss: 1.5434 - acc: 0.4395 - val_loss: 1.7376 - val_acc: 0.4028
Epoch 6/15
421/421 [==============================] - 131s 311ms/step - loss: 1.4629 - acc: 0.4733 - val_loss: 1.5790 - val_acc: 0.4347
Epoch 7/15
421/421 [==============================] - 131s 311ms/step - loss: 1.3844 - acc: 0.5023 - val_loss: 1.5370 - val_acc: 0.4753
Epoch 8/15
421/421 [============================

In [66]:
accuracy = model1.evaluate(test_generator)

print("Accuracy based on simple Dense Model :- {:.2f}%".format(accuracy[1]*100))

4492/4492 [==============================] - 186s 42ms/step - loss: 1.1901 - acc: 0.5828
Accuracy based on simple Dense Model :- 58.28%


## Second Model -> CNN Model

In [67]:
train_generator, val_generator, test_generator = imagedatageneration(train_dir, val_dir, test_dir)

Found 13451 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.
Found 4492 images belonging to 10 classes.


In [68]:
model2 = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation = 'relu', input_shape = (256, 256, 3)),
    Conv2D(32, (3, 3), activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation = 'relu'),
    Conv2D(64, (3, 3), activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation = 'relu'),
    BatchNormalization(),
    Flatten(),
    Dense(512, activation = 'relu'),
    BatchNormalization(),
    Dense(256, activation = 'relu'),
    BatchNormalization(),
    Dense(256, activation = 'relu'),
    BatchNormalization(),
    Dense(128, activation = 'relu'),
    BatchNormalization(),
    Dense(10, activation = 'softmax')
])

In [70]:
model2.compile(optimizer = Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['acc'])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 32)      9248      
                                                                 
 batch_normalization_6 (Bat  (None, 252, 252, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 126, 126, 32)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 122, 122, 64)     

In [71]:
model2.fit(train_generator,
          epochs = 15,
          verbose = 1,
          validation_data = val_generator,
          callbacks = [es])

Epoch 1/15
421/421 [==============================] - 491s 1s/step - loss: 1.6521 - acc: 0.4232 - val_loss: 2.0469 - val_acc: 0.3517
Epoch 2/15
421/421 [==============================] - 487s 1s/step - loss: 1.0345 - acc: 0.6482 - val_loss: 1.0111 - val_acc: 0.6474
Epoch 3/15
421/421 [==============================] - 475s 1s/step - loss: 0.7023 - acc: 0.7634 - val_loss: 1.5528 - val_acc: 0.5200
Epoch 4/15
421/421 [==============================] - 480s 1s/step - loss: 0.5491 - acc: 0.8157 - val_loss: 0.5341 - val_acc: 0.8199
Epoch 5/15
421/421 [==============================] - 478s 1s/step - loss: 0.4361 - acc: 0.8537 - val_loss: 0.5781 - val_acc: 0.8076
Epoch 6/15
421/421 [==============================] - 485s 1s/step - loss: 0.3842 - acc: 0.8712 - val_loss: 0.3223 - val_acc: 0.8880
Epoch 7/15
421/421 [==============================] - 482s 1s/step - loss: 0.3314 - acc: 0.8918 - val_loss: 0.5340 - val_acc: 0.8183
Epoch 8/15
421/421 [==============================] - 480s 1s/step - 

In [72]:
accuracy = model2.evaluate(test_generator)

print("Accuracy based on our CNN Model :- {:.2f}%".format(accuracy[1]*100))

4492/4492 [==============================] - 134s 30ms/step - loss: 0.1879 - acc: 0.9437
Accuracy based on our CNN Model :- 94.37%


## Third Model -> VGG16

In [90]:
train_generator, val_generator, test_generator = imagedatageneration(train_dir, val_dir, test_dir)

Found 13451 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.
Found 4492 images belonging to 10 classes.


In [91]:
pretrained_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))
pretrained_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [92]:
for layer in pretrained_model.layers[:-5]:
    layer.trainable = False
    

In [93]:
# last_layer = pretrained_model.get_layer('block5_pool')
# last_output = last_layer.output

model3 = tf.keras.models.Sequential([ 
    pretrained_model,
    Flatten(),
    Dense(512, activation = 'relu'),
    BatchNormalization(),
    Dense(256, activation = 'relu'),
    BatchNormalization(),
    Dense(128, activation = 'relu'),
    BatchNormalization(),
    Dense(10, activation = 'softmax')
    
])

In [94]:
model3.compile(optimizer = Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['acc'])
model3.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_6 (Flatten)         (None, 32768)             0         
                                                                 
 dense_25 (Dense)            (None, 512)               16777728  
                                                                 
 batch_normalization_20 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dense_26 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_21 (Ba  (None, 256)               1024      
 tchNormalization)                                    

In [95]:
model3.fit(train_generator,
          epochs = 15,
          verbose = 1,
          validation_data = val_generator,
          callbacks = [es])


# model3.fit(train_generator,
#           steps_per_epoch = 250,
#           epochs = 20,
#           verbose = 1,
#           validation_steps = 50,
#           validation_data = val_generator,
#           callbacks = [es])

Epoch 1/15
421/421 [==============================] - 1341s 3s/step - loss: 0.9758 - acc: 0.6730 - val_loss: 1.6045 - val_acc: 0.6628
Epoch 2/15
421/421 [==============================] - 1319s 3s/step - loss: 0.3298 - acc: 0.8918 - val_loss: 0.7769 - val_acc: 0.7563
Epoch 3/15
421/421 [==============================] - 1318s 3s/step - loss: 0.1872 - acc: 0.9409 - val_loss: 0.4495 - val_acc: 0.8525
Epoch 4/15
421/421 [==============================] - 1318s 3s/step - loss: 0.1743 - acc: 0.9457 - val_loss: 0.2425 - val_acc: 0.9232
Epoch 5/15
421/421 [==============================] - 3260s 8s/step - loss: 0.1477 - acc: 0.9525 - val_loss: 1.5301 - val_acc: 0.6217
Epoch 6/15
421/421 [==============================] - 1823s 4s/step - loss: 0.1285 - acc: 0.9616 - val_loss: 0.8088 - val_acc: 0.7717


In [96]:
accuracy = model3.evaluate(test_generator)

print("Accuracy based on our VGG16 Model :- {:.2f}%".format(accuracy[1]*100))

4492/4492 [==============================] - 381s 85ms/step - loss: 0.7634 - acc: 0.7892
Accuracy based on our VGG16 Model :- 78.92%


## Fourth Model -> ResNet50

In [83]:
train_generator, val_generator, test_generator = imagedatageneration(train_dir, val_dir, test_dir)

Found 13451 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.
Found 4492 images belonging to 10 classes.


In [84]:
pretrained_model = ResNet50(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))

In [85]:
for layer in pretrained_model.layers[:-3]:
    layer.trainable = False

In [86]:
model4 = tf.keras.models.Sequential([ 
    pretrained_model,
    Flatten(),
    Dense(512, activation = 'relu'),
    BatchNormalization(),
    Dense(256, activation = 'relu'),
    BatchNormalization(),
    Dense(128, activation = 'relu'),
    BatchNormalization(),
    Dense(10, activation = 'softmax')
])

In [87]:
model4.compile(optimizer = Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['acc'])
model4.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten_5 (Flatten)         (None, 131072)            0         
                                                                 
 dense_21 (Dense)            (None, 512)               67109376  
                                                                 
 batch_normalization_17 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dense_22 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_18 (Ba  (None, 256)               1024      
 tchNormalization)                                    

In [88]:
model4.fit(train_generator,
          epochs = 15,
          verbose = 1,
          validation_data = val_generator,
          callbacks = [es])


# model4.fit(train_generator,
#           steps_per_epoch = 250,
#           epochs = 20,
#           verbose = 1,
#           validation_steps = 50,
#           validation_data = val_generator,
#           callbacks = [es])

Epoch 1/15
421/421 [==============================] - 430s 1s/step - loss: 1.7425 - acc: 0.3945 - val_loss: 3.1864 - val_acc: 0.2593
Epoch 2/15
421/421 [==============================] - 430s 1s/step - loss: 1.1782 - acc: 0.5933 - val_loss: 1.1323 - val_acc: 0.6025
Epoch 3/15
421/421 [==============================] - 464s 1s/step - loss: 0.9540 - acc: 0.6779 - val_loss: 0.9264 - val_acc: 0.6791
Epoch 4/15
421/421 [==============================] - 642s 2s/step - loss: 0.8344 - acc: 0.7136 - val_loss: 0.9784 - val_acc: 0.6809
Epoch 5/15
421/421 [==============================] - 970s 2s/step - loss: 0.7440 - acc: 0.7494 - val_loss: 0.7869 - val_acc: 0.7228
Epoch 6/15
421/421 [==============================] - 412s 978ms/step - loss: 0.6847 - acc: 0.7671 - val_loss: 0.9296 - val_acc: 0.6967
Epoch 7/15
421/421 [==============================] - 416s 989ms/step - loss: 0.6348 - acc: 0.7869 - val_loss: 0.7000 - val_acc: 0.7581
Epoch 8/15
421/421 [==============================] - 494s 1s/s

In [89]:
accuracy = model4.evaluate(test_generator)

print("Accuracy based on our ResNet50 Model :- {:.2f}%".format(accuracy[1]*100))

4492/4492 [==============================] - 352s 78ms/step - loss: 0.5178 - acc: 0.8257
Accuracy based on our ResNet50 Model :- 82.57%


## Fifth Model -> Yolo v8

In [ ]:
model5 = YOLO('yolov8n-cls.pt')

In [ ]:
# splitfolders.ratio("../datasets/state-farm-distracted-driver-detection/imgs/train", output="../datasets/state-farm-distracted-driver-detection/output", seed = 1337, ratio=(0.7, 0.15, 0.15))

In [ ]:
results = model5.train(data = "../datasets/state-farm-distracted-driver-detection/output", epochs = 10)

In [ ]:
model5.val()

In [ ]:
df= pd.read_csv('./runs/classify/train2/results.csv')
df.head()

In [ ]:
Image("./runs/classify/train2/results.png")

In [ ]:
Image("./runs/classify/train2/confusion_matrix_normalized.png")


In [ ]:
# classes = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
# path = "../datasets/state-farm-distracted-driver-detection/output/test/c1/"
# actual_class = 1
# model_weights = "./runs/classify/train2/weights/best.pt"
# pred = [(path+i,model5.predict(path+i, model = model_weights)[0].probs.top1, actual_class) for i in os.listdir(path)[:45]]

In [ ]:
classes = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
test_images_path = "../datasets/state-farm-distracted-driver-detection/output/test/"
model_weights = "./runs/classify/train2/weights/best.pt"

predicted_list = []

for clas in classes:
    image_dir = os.path.join(test_images_path, clas)
    # print(image_dir)
    images_list = os.listdir(image_dir)
    # print(images_list)
    # Class label in the form of 0 to 9
    class_label = int(clas[-1])
    # print(class_label)
    for image in images_list:
        path = os.path.join(image_dir, image)
        # print(path)
        y_actual = class_label
        y_predicted = model5.predict(path, model = model_weights)[0].probs.top1
        predicted_list.append([path, y_actual, y_predicted])

In [ ]:
print("length of the Predicted List : ", len(predicted_list))

In [ ]:
df = pd.DataFrame(predicted_list, columns = ['Image_path', 'Y_actual', 'Y_predicted'])

In [ ]:
accuracy_score(df['Y_actual'], df['Y_predicted'])